In [6]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:

from util import *
import numpy as np
import sys
import sklearn.cluster as clst
import sklearn.mixture as mix
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pickle
import random as rd

In [8]:
def birch_clustering(nodes, eps, n_clst = None):
    # Fitting 
    batch_len = 1000
    Birch_clst = clst.Birch(eps, n_clusters = None)
    Birch_labels = []
    for i in range(len(nodes)//batch_len):
        Birch_clst.partial_fit(nodes[i*batch_len:(i+1)*batch_len])
    Birch_clst.partial_fit(nodes[(len(nodes)//batch_len)*batch_len:])
    
    Birch_clst.set_params(n_clusters = n_clst)
    print("Begin global fit")
    Birch_clst.partial_fit()
    
    print("Begin global labeling")
    Birch_labels = Birch_clst.predict(nodes)
    print("max_label: ", max(Birch_labels))
    
    # Data collecting
    Birch_stats, Birch_out = clst_stats(Birch_labels)

    #out_liers
    if n_clst is None:
        Birch_out = Birch_stats[1]
        Birch_stats[1] = 0
    else:
        Birch_stats[1] = 0
        Birch_out = n_clst - sum(Birch_stats)
    Birch_error = SSE(nodes, Birch_labels, Birch_clst.subcluster_centers_)

    summa = sum([i*Birch_stats[i] for i in range(len(Birch_stats))])
    print("Number of nodes getting clustered: ", summa)
    print("Number of clusters: ", sum(Birch_stats))
    print("Number of outliers: ", Birch_out)
    print("MSE: ", Birch_error/summa)
    
    return Birch_labels, Birch_stats, Birch_out, Birch_error/summa

In [11]:
#TOFDO FOR RESUME:
# 1. change Config["filenames"] to only desired targets
# 2. update n_clsts in nxt block 
# 3. switch to "if i == -1:"

Config = {}
Config["directory"] = "./data/"
Config["folders"] = ["9-1"]
Config["filenames"] = ["real-data", "data-max", "data-middle" ,"data-min","data-uniform"]
epsilons = [0.04, 0.05, 0.06, 0.07, 0.08]
figsizes = [50, 200]
np.set_printoptions(suppress=True)
#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(threshold=100)

In [1]:
for k in range(len(Config["folders"])):
    
    folder = Config["folders"][k]
    n_clsts = []
    
    #n_clsts = [8154, 5265, 4158, 2992, 2388]
    #n_clsts = [9790, 6791, 5057, 4000, 3046]
    #n_clsts = [10038, 7117, 5170, 4325, 3650]
    #n_clsts = [10059, 7139, 5125, 3964, 3037]
    #n_clsts = [6596, 4758, 3525, 2751, 2238]
    #n_clsts = [9189, 6003, 4481, 3336, 2659]
    statsss = []
    results = []

    for i in range(len(Config["filenames"])):

        filename = Config["filenames"][i]

        #retrieve data from file
        print("Reading data from and generating Birch results for %s:"%filename)
        x,y = read_data(Config["directory"] + folder + "/"+ filename + ".xlsx")

        # transform data
        nodes = np.asarray([x,y]).T
        print(nodes[0:3])
        np.random.shuffle(nodes)

        print("Number of nodes: ", len(nodes))

        # Do birchs
        result = []
        statss = []
        n_clst = 0

        for j in range(len(epsilons)):

            # Do birch 
            epsilon = epsilons[j]
            print("Epsilon = ", epsilon, " ###################################")
            
            #if i == -1: turn on if resumed from checkpoint
            if i == 0:
                labels, stats, out, MSE = birch_clustering(nodes, epsilon)
                n_clst = int(out + sum(stats))
                n_clsts.append(n_clst)
                print("Birch estimated cluster number: ", n_clst)
            else: 
                n_clst = n_clsts[j]
                labels, stats, out, MSE = birch_clustering(nodes, epsilon, n_clst)
            result.append((labels, stats, out, MSE))
            stats[1] = out
            statss.append(stats)
            stats[1] = 0

            plotter(stats, epsilon, "birch", folder + "_" + filename, 0, 100, n_clst)
            weighted_plotter(stats, epsilon, "birch", folder + "_" + filename, 0, 100, n_clst)
            for figsize in figsizes:
                plot_graph(nodes[:,0], nodes[:,1], epsilon, n_clst, labels, "Birch", folder+"_"+filename, figsize)

        # checkpoint  
        results.append(result)
        statsss.append(statss)

        BIRCH = (n_clst, result, Config["filenames"], epsilons, statss)
        file = open("./BIRCH-%s-%s.tar"%(folder,filename),"wb")
        pickle.dump(BIRCH, file)
        file.close()


NameError: name 'Config' is not defined